# Generating Irony data

**Table of contents**:
- Qwen/Qwen2.5-0.5B-Instruct
    1. Baseline
    2. Targeted
    3. Targeted + Linguistic tags
- deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
    1. Baseline
    2. Targeted
    3. Targeted + Linguistic tags

**Results**:
- irony seems a too difficult topic for this models, the "irony" sentences aren't ironic (also the "targeted" prompts case).
- doesn't follow always the instruction and is very sensitive to prompt, but it follows nicely the JSON format.
- ChatGPT gives examples that are actually ironic.

In [ ]:
import os

# Move up one directory
if os.path.basename(os.getcwd()) == "mycode":
    os.chdir("..")
    
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import re
import json
from mycode.utilities import log_synthetic_data, response2json, get_response, set_seed, clear_cuda_cache

# Create the folder to save the synthetic data
folder_name = "synthetic_data"
os.makedirs(folder_name, exist_ok=True)

# file where the logs will be saved
log_file_path = "synthetic_data/semevalirony_log.json"
if os.path.exists(log_file_path):
    os.remove(log_file_path)

# device
device = 'cuda:0'

# DATA
df = pd.read_csv("LREC-COLING/train/semevalironytrainAll.csv")
df = df.rename(columns={"1": "text", "2": "label"})
display(df.head())
first_irony = df[df['label'] == 'irony'].iloc[0].loc['text']
first_non_irony = df[df['label'] == 'non-irony'].iloc[0].loc['text']

print("\nFirst irony example:")
print(first_irony)

print("\nFirst non-irony example:")
print(first_non_irony)

,0,text,label
0,0,seeing ppl walking w/ crutches makes me really...,irony
1,1,"look for the girl with the broken smile, ask h...",non irony
2,2,Now I remember why I buy books online @user #s...,irony
3,3,@user @user So is he banded from wearing the c...,irony
4,4,Just found out there are Etch A Sketch apps. ...,irony



First irony example:
seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life

First non irony example:
look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵


# Qwen/Qwen2.5-0.5B-Instruct
## 1. Baseline

In [2]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# for qwen we use this system prompt:
system_prompt_qwen = "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."

In [ ]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for the examples:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 1.16
GENERATED RESPONSE:
I'm ready to assist you with your request! Please provide me with the text and label for each example.


In [ ]:
# With example for the two classes
prompt =f"""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non-irony
"""
generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 0.85
GENERATED RESPONSE:
text: Seeing people walking with crutches makes me super excited about the next three weeks of my life.
Label: irony

text: Look for the girl with the broken smile, ask her if she wants to stay, and she will be loved. 🕺💖
Label: non irony


In [ ]:
# act as a linguist and NLP practitioner
prompt =f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model

Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 5.07
GENERATED RESPONSE:
Sure! Here are ten examples that I've generated to detect irony in written text:

### Irony Example 1
> "I was so excited about my new job that I couldn't contain myself and burst into tears."

**Label:** Irony

### Irony Example 2
> "The cat chased the mouse but the mouse didn’t care because it knew the cat would eat it."

**Label:** Irony

### Irony Example 3
> "He said he would go on a vacation next week, but he never showed up at all."

**Label:** Irony

### Irony Example 4
> "She told me she would come over for dinner, but when she arrived, she wasn't even there."

**Label:** Irony

### Irony Example 5
> "The judge ruled that the defendant should be sentenced to life imprisonment."

**Label:** Irony

### Irony Example 6
> "The politician promised to vote for the bill, but when they voted, they changed their minds."

**Label:** Irony

### Irony Example 7
> "The teacher explained the rules of the game, but the players didn't understand."

**Label

In [ ]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of "irony" and "non-irony" statements, \
with **5 irony** and **5 non-irony** examples across different contexts.

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "irony"}},
    {{"text": "{first_non_irony}", "label": "non-irony"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 2.91
GENERATED RESPONSE:
Here are ten examples of "irony" and "non-irony" statements with five each:

```json
[
    {
        "text": "I'm so hungry, but I don't have any food at home.",
        "label": "irony"
    },
    {
        "text": "The sun was shining and I decided to go for a walk.",
        "label": "non-irony"
    },
    {
        "text": "He's going on a date with someone new, and he looks really good.",
        "label": "irony"
    },
    {
        "text": "The sky is blue today, but it's raining.",
        "label": "non-irony"
    },
    {
        "text": "I forgot about my appointment, but I'm not late.",
        "label": "irony"
    },
    {
        "text": "She just told me she's going on a trip next week.",
        "label": "non-irony"
    }
]
```


In [7]:
synthetic_data = response2json(response)

print("synthetic_data[:2]: ", synthetic_data[:2])

synthetic_data[:2]:  [{'text': "I'm so hungry, but I don't have any food at home.", 'label': 'irony'}, {'text': 'The sun was shining and I decided to go for a walk.', 'label': 'non-irony'}]


In [8]:
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

# Load JSON file
with open(log_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Print the first log entry
print(json.dumps(data[:1], indent=4, ensure_ascii=False))  # Pretty-print the first entry

Logged 6 examples to synthetic_data/semevalirony_log.json. Time taken: 2.91 seconds
[
    {
        "timestamp": "2025-03-10T12:41:44.032109",
        "model": "Qwen/Qwen2.5-0.5B-Instruct",
        "generation_method": "baseline",
        "prompt": "You are an expert linguist and NLP practitioner specializing in irony detection. Your task is to generate **10 high-quality examples** of \"irony\" and \"non irony\" statements, with **5 irony** and **5 non irony** examples across different contexts.\n\n### **Output Format (JSON)**\nReturn only a valid JSON list in the following structure:\n\n```json\n[\n    {\"text\": \"seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life\", \"label\": \"irony\"},\n    {\"text\": \"look for the girl with the broken smile, ask her if she wants to stay while, and she will be loved. 💕🎵\", \"label\": \"non irony\"},\n    ...\n]\n```\n",
        "time_taken_seconds": 2.91,
        "num_examples": 6,
        "generated_examples"

In [9]:
# Convert log entries to a DataFrame
df = pd.DataFrame(data)

# Expand the 'generated_examples' column
samples_df = df.explode("generated_examples").reset_index(drop=True)

# Convert 'generated_examples' (which is still a dictionary) into separate columns
samples_df = pd.concat([samples_df.drop(columns=["generated_examples"]), samples_df["generated_examples"].apply(pd.Series)], axis=1)

display(samples_df.head())

,timestamp,model,generation_method,prompt,time_taken_seconds,num_examples,text,label
0,2025-03-10T12:41:44.032109,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,2.91,6,"I'm so hungry, but I don't have any food at home.",irony
1,2025-03-10T12:41:44.032109,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,2.91,6,The sun was shining and I decided to go for a ...,non-irony
2,2025-03-10T12:41:44.032109,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,2.91,6,"He's going on a date with someone new, and he ...",irony
3,2025-03-10T12:41:44.032109,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,2.91,6,"The sky is blue today, but it's raining.",non-irony
4,2025-03-10T12:41:44.032109,Qwen/Qwen2.5-0.5B-Instruct,baseline,You are an expert linguist and NLP practitione...,2.91,6,"I forgot about my appointment, but I'm not late.",irony


## 2. Targeted synthetic data

In [10]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be covered by a irony detection model to perform accurately. \
Provide concrete examples illustrating each phenomenon. \
"""
response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 13.87
GENERATED RESPONSE:
Certainly! Here is a comprehensive list of key linguistic and semantic phenomena that need to be covered for an ironic detection model:

### Linguistic Phenomena

1. **Word Choice**:
   - **Example**: "The quick brown fox jumps over the lazy dog."
     - This sentence uses hyperbole ("quick" being very fast) and metaphorical language ("jumps over the lazy dog") to convey sarcasm.

2. **Phonological Variation**:
   - **Example**: "She said 'I'm so hungry.'"
     - The word order in this sentence (e.g., "I'm so hungry.") is inverted, which can indicate sarcasm or misinterpretation.

3. **Pronunciation and Spelling**:
   - **Example**: "It's raining cats and dogs outside."
     - The spelling error ("cats and dogs" instead of "cats and dogs") could be a form of irony, suggesting a misunderstanding or exaggeration.

4. **Sentence Structure**:
   - **Example**: "He ate the cake without even knowing it was there."
     - The lack of clarity in the senten

In [11]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these phenomena as a structured list, without explanations. \
For example: sarcasm, negation, pragmatic inference, unexpected contrast. \
Return the list in a simple bullet-point format.\
"""
response, delta_t_0 = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)

TIME TAKEN: 0.4
GENERATED RESPONSE:
- Irony Detection
- Semantic Analysis
- Contrast Detection
- Pragmatic Inference
- Negation
- Sarcasm


In [ ]:
# Replace "- " at the beginning of each line with a comma
csv_string = re.sub(r"^\s*-\s*", "", response, flags=re.MULTILINE)  # Remove bullet points
csv_string = ", ".join(csv_string.strip().split("\n"))  # Join lines with ", "

prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **irony**.
- 5 sentences are **non-irony**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "irony"}},
    {{"text": "{first_non_irony}", "label": "non-irony"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
# synthetic_data = response2json(response)
# log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 4.24
GENERATED RESPONSE:
```json
[
    {
        "text": "The cat was sleeping on the couch when the dog walked over it.",
        "label": "irony"
    },
    {
        "text": "He said he would meet you at the park but then he forgot about it.",
        "label": "non irony"
    },
    {
        "text": "I had to leave early because I was running late, so I decided to go home.",
        "label": "irony"
    },
    {
        "text": "She said she wanted to go to the party but then realized she didn't have any money.",
        "label": "non irony"
    },
    {
        "text": "The company is going through a difficult time, so they need all the help they can get.",
        "label": "irony"
    },
    {
        "text": "The movie was terrible, but I still enjoyed it because I'm a big fan.",
        "label": "non irony"
    },
    {
        "text": "The sun was shining brightly today, so everyone went outside for a picnic.",
        "label": "irony"
    },
    {
        "text": 

### Targeted synthetic data with Chat-GPT

The topics that need to be covered are generated from Chat-GPT online.

In [13]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
Generate 10 realistic sentences illustrating irony detection examples involving.
For each example specify the label as either "irony" or "non-irony".

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <label>}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 3.99
GENERATED RESPONSE:
```json
[
    {
        "text": "The cat is everywhere.",
        "label": "Irony"
    },
    {
        "text": "I love ice cream, but I don't like chocolate.",
        "label": "Non-irony"
    },
    {
        "text": "She always says she'll be here tomorrow, but she's actually going somewhere else.",
        "label": "Irony"
    },
    {
        "text": "We should eat more vegetables, but we're having too many meals.",
        "label": "Non-irony"
    },
    {
        "text": "He said he would come over tonight, but he didn't.",
        "label": "Irony"
    },
    {
        "text": "This movie is great, but it's not really what I expected.",
        "label": "Non-irony"
    },
    {
        "text": "They said they'd go on vacation next year, but they haven't decided yet.",
        "label": "Irony"
    },
    {
        "text": "I can't believe you did that, but you're still doing it.",
        "label": "Non-irony"
    },
    {
        "text": "He p

## 3. Targeted + Tags linguistic phenomena 

Now we ask to the model also to identify the linguistic phenomena present in the generated sentence.

```
{  
    "text": "Oh, I absolutely adore being stuck in traffic for hours.",   
    "label": "ironic",  
    "phenomena": ["Polarity inversion", "Hyperbole", "Semantic incongruence", "Lexical exaggeration"]
},
```

In [14]:
prompt = f"""\
You are an expert linguist and NLP specialist in sarcasm and irony detection. \
Generate 10 realistic sentences illustrating irony detection examples.\
For each example specify the label as either "irony" or "non-irony". And also list the key phenomena it covers.

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <corresponding label>, "phenomena": ["<phenomenon1>", "<phenomenon2>", ...]}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer, system_prompt=system_prompt_qwen)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted + linguistic tags", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 5.32
GENERATED RESPONSE:
```json
[
    {
        "text": "The company's CEO announced they would be launching a new product.",
        "label": "non-irony",
        "phenomena": ["Lexical Choice", "Punctuation"]
    },
    {
        "text": "I'm so tired of this boring lecture.",
        "label": "non-irony",
        "phenomena": ["Negation", "Syntactic Cues"]
    },
    {
        "text": "She said she was going on vacation tomorrow.",
        "label": "non-irony",
        "phenomena": ["Negation", "Sarcasm"]
    },
    {
        "text": "He said he couldn't make it to the party.",
        "label": "non-irony",
        "phenomena": ["Negation", "Hyperbole & Understatement"]
    },
    {
        "text": "They will have to work hard to finish the project.",
        "label": "non-irony",
        "phenomena": ["Contextual Incongruity"]
    },
    {
        "text": "It is raining heavily outside.",
        "label": "non-irony",
        "phenomena": ["Contextual Incongruity"]
   

---

# deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

## 1. Baseline

In [15]:
clear_cuda_cache(model)

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2',
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.generation_config.pad_token_id = tokenizer.pad_token_id

# we don't submit a system prompt as suggested in the model card

In [ ]:
prompt ="""\
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for the examples:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer)

TIME TAKEN: 16.89
GENERATED RESPONSE:
Okay, so the user has asked me to produce 10 examples of irony in written text, categorizing them into 5 "irony" and 5 "non-irony" examples. They also provided a specific format for each example, which includes the text and a label.

First, I need to make sure I understand what irony is. Irony is when the meaning of a statement is the opposite of its literal meaning. It's often used to express sarcasm or surprise. So, I need to come up with both ironic and non-ironic examples.

For the ironic examples, I should think of situations where the punchline or implication is the opposite of what's stated. Maybe a situation where the speaker is saying something that's contrary to what's true, or where the situation is the opposite of what's expected.

Let me brainstorm some scenarios. Maybe a situation where something is said that's contrary to fact, like a lie being told truthfully. Or a situation where the opposite of what's expected happens. For example

In [ ]:
# With example for the two classes
prompt =f"""\
Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples  for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: {first_irony}
label: irony

text: {first_non_irony}
label: non-irony\
"""
generated, delta_t = get_response(prompt, model, tokenizer)

TIME TAKEN: 35.45
GENERATED RESPONSE:
Okay, so I need to generate 10 examples of synthetic text for improving an irony detection model. The task is to create exactly 5 irony examples and 5 non-ironicity examples. The format should be something like "text: [example text] label: [ironicity label]."

First, I should understand what irony is. Irony is when the meaning of something is opposite to what is implied. It often involves a situation where the punchline is the opposite of what is actually happening. For example, "What do you mean when you say I'm good at cooking?" is ironic because it's a play on words.

I need to create examples that are either ironic or non-ironic. The user provided two examples: one labeled as irony and one as non-ironicity. Let me look at those.

In the first example, "seeing ppl walking w/ crutches makes me really excited for the next 3 weeks of my life" is labeled as irony. The context is about someone excited about the future, and the action (walking with cr

In [18]:
# act as a linguist and NLP practitioner
prompt =f"""\
<context>
You are an expert linguist and NLP practitioner specializing in irony detection.
</context>

Your task is to generate high-quality synthetic examples of ironic and non-ironic text to improve an irony detection model. \
Produce 10 examples for detecting irony in written text. Examples are categorized as either "irony" or "non-irony". \
Produce exactly 5 examples for "irony" and 5 examples for "non-irony". 

Use this format for generating the data:
text: <text>
label: <label>
"""
generated, delta_t = get_response(prompt, model, tokenizer)

TIME TAKEN: 22.54
GENERATED RESPONSE:
Okay, so I need to create 10 examples of irony and non-ironic text for a model that detects irony. I have to make exactly 5 for each category. Let me think about how to approach this.

First, I should understand what irony is. Irony is when the meaning of a statement is opposite of what it actually says. It often involves contradiction, surprise, or surprise in the other way. For example, "I love you, but I hate you" is ironic because it's contradictory.

I need to come up with different scenarios where the irony is clear. Let me brainstorm some situations where irony is evident.

For the irony examples, I can think of situations where a statement is true, but the speaker is implying it's false, or vice versa. Maybe some scenarios with emotions involved, like someone being surprised but saying the opposite of what they're feeling.

Let's start with the irony examples:

1. The sun setting while it's still rising. Wait, that's a bit abstract. Maybe a

In [19]:
prompt = f"""\
You are an expert linguist and NLP practitioner specializing in irony detection. \
Your task is to generate **10 high-quality examples** of ironic and non-ironic statements, \
with **5 ironic** and **5 non-ironic** examples across different contexts.

### **Output Format (JSON)**
Return the final result into a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "baseline", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 20.04
GENERATED RESPONSE:
Okay, so I need to create 10 examples of ironic and non-ironic statements, with exactly 5 of each. The user has already provided an example, so I need to make sure I follow that format. Let me think about how to approach this.

First, I should understand what makes a statement ironic. Irony occurs when the meaning of the words is opposite to what is implied, often due to the nature of the words or the context. So, I need to identify statements where the effect contradicts the intention or where the situation is inverted.

For the ironic examples, I want to look for situations where the words imply something contrary to what is actually meant. Maybe using words that suggest a negative outcome when they shouldn't, or positive when they should be negative, or vice versa.

Let me brainstorm some scenarios where irony is common. For example, "I see a crocodile crossing the road" is ironic because the presence of a crocodile implies danger, which is the 

## 2. Targeted synthetic data


In [20]:
prompt = f"""\
You are an expert linguistics and NLP practitioner. \
List all key linguistic and semantic phenomena that must be considered for an irony detection model to perform accurately. \
Provide only the names of these key concepts as a structured list, without explanations.

Return the list in a simple bullet-point format, each bullet point must start with a dash "-".\
"""

response, delta_t_0 = get_response(prompt, model, tokenizer)

# Extract bullet points
bullets = re.findall(r"- (.+)", response)
bullets = [b.strip() for b in bullets]  # Remove leading/trailing whites

# Convert to CSV string
csv_string = ", ".join(bullets)

TIME TAKEN: 13.82
GENERATED RESPONSE:
Okay, so I need to figure out the key linguistic and semantic phenomena that are important for detecting irony in text. I'm a bit new to this, so I'll start by breaking down what irony is. Irony is when the intended meaning of two statements is opposite, often because of wordplay or ambiguity. Examples include "Why are you here?" and "Why are you leaving?" which are contradictory.

First, I should think about the grammatical aspects. Maybe things like question words, statements, and how they relate to each other. For instance, if someone is asking a question and then says "I'm done," that's irony because the question and response are contradictory.

Then, there's the concept of contradiction. When two statements are opposites. But that's too broad. I need more specific terms. Maybe the relationship between the subject and predicate in each statement. If the subject is the same but the predicate is contradictory, that's a clue.

I also remember some

In [21]:
prompt = f"""\
Generate 10 realistic sentences illustrating irony detection examples involving {csv_string}. Ensure that:
- 5 sentences are **ironic**.
- 5 sentences are **non-ironic**.

### **Output Format (JSON)**
Return **only** a valid JSON list in the following structure:

```json
[
    {{"text": "{first_irony}", "label": "ironic"}},
    {{"text": "{first_non_irony}", "label": "non-ironic"}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
# synthetic_data = response2json(response)
# log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 14.32
GENERATED RESPONSE:
Okay, so the user has asked me to generate 10 sentences that illustrate different types of irony. They provided a clear structure with 5 ironic and 5 non-ironic sentences. I need to make sure I understand each type of irony they're looking for.

First, I'll break down each category. For ironic sentences, I should focus on situations where the irony is direct and surprising. Maybe something about walking with chutes, as they mentioned. Then, inverted question structures, so I'll need sentences where the question is phrased backwards but still makes sense.

Contrary clauses are sentences where the usual structure is flipped. I'll have to think of scenarios where the opposite of what's stated makes sense. Paradoxical relationships could involve situations where two unrelated actions lead to a surprising conclusion, like a car breaking down and being repaired.

Subject-predicate inversion is a bit tricky. I'll need to switch the subject and predicate i

### Targeted synthetic data with Chat-GPT

In [29]:
prompt = f"""\
You are an expert linguist and NLP practitioner. \
Generate 10 realistic sentences illustrating irony detection examples.\
For each example specify the label as either "irony" or "non-irony".

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
For each example specify the label as either "irony" or "non-irony".
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <label>}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 19.94
GENERATED RESPONSE:
Okay, so I need to generate 10 realistic sentences that illustrate irony using the provided phenomena. Let me break down the requirements.

First, I should understand the phenomena: linguistic, semantic, and contextual. The user provided four categories for each, so I'll need to pick one category for each example. 

Linguistic Phenomena include unusual word choice, negation, punctuation, complex sentence structures, and contrastive conjunctions. The semantic phenomena are contextual incongruity, polarity reversal, hyperbole, and sarcasm. Contextual cues are world knowledge, speaker intent, and discourse contrast.

I need to create sentences that fit one of these categories. Let's brainstorm a variety of examples.

1. **Contrastive Conjunctions**: Maybe a statement that reverses the obvious. Like saying "I love him, but he loves me" using "but" to contradict. So "I love him, but he loves me. You should never be in his presence." Here, "but" is a con

## 3. Targeted + Tags linguistic phenomena 

In [23]:
prompt = f"""\
You are an expert linguist and NLP specialist in sarcasm and irony detection. \
Generate 10 realistic sentences illustrating irony detection examples.\
For each example specify the label as either "irony" or "non-irony", give examples for both of them. And also list the key phenomena it covers.

### **Consider the following Phenomena:**
- **Linguistic Phenomena**  
    - Lexical Choice: Unusual or exaggerated word use.  
    - Negation: Statements that negate obvious facts.  
    - Punctuation: Use of exclamation marks, ellipses, or quotes for emphasis.  
    - Syntactic Cues: Unusual or complex sentence structures.  
    - Contrastive Conjunctions: Use of "but," "however," to signal contradiction.  

- **Semantic Phenomena**  
    - Contextual Incongruity: Discrepancy between words and context.  
    - Polarity Reversal: Positive words with negative intent, or vice versa.  
    - Hyperbole & Understatement: Exaggeration or minimization for effect.  
    - Sarcasm: Mocking statements implying the opposite meaning.  

- **Contextual Cues**  
    - World Knowledge: Understanding cultural or situational references.  
    - Speaker Intent: Inferring the true intention behind words.  
    - Discourse Contrast: Contradictions across multiple sentences.  

### **Output Format (JSON)**
Return only a valid JSON list in the following structure:

```json
[
    {{"text": <text>, "label": <corresponding label>, "phenomena": ["<phenomenon1>", "<phenomenon2>", ...]}},
    ...
]
```
"""
response, delta_t = get_response(prompt, model, tokenizer)
synthetic_data = response2json(response)
log_synthetic_data(model_name, "targeted + linguistic tags", prompt, synthetic_data, delta_t, output_file=log_file_path)

TIME TAKEN: 24.04
GENERATED RESPONSE:
Okay, so I need to create 10 examples of irony and non-irony sentences based on the provided phenomena. Let me first make sure I understand the criteria for each category.

Starting with the label "irony." I know irony is when the meaning of the words is opposite to their literal meaning. Examples could involve exaggerated words, negations, complex sentences, or contextually inconsistent statements.

For "non-irony," it's when the irony is subtle or comes from context or word choice that doesn't change the literal meaning. Maybe using a word without the right connotation or a situation where the irony is implied.

Now, looking at the phenomena:

1. **Lexical Choice**: Using unusual words. For irony, maybe a word that's not commonly used but conveys strong emotions.
2. **Negation**: Statements that contradict obvious facts. For example, "I don't know if I'm right."
3. **Punctuation**: Using exclamation marks or ellipses to emphasize. Irony might com

### Example of data generated by Chat-GPT

"text": "Oh, great! Another rainy day for my outdoor event. Just what I needed!",  
"label": "irony",  
"phenomena": ["Punctuation", "Contextual Incongruity", "Sarcasm"]  

"text": "I love waiting in long lines at the grocery store. It’s my favorite pastime!",  
"label": "irony",  
"phenomena": ["Lexical Choice", "Sarcasm"]  

"text": "I’m really excited to spend the weekend doing absolutely nothing.",  
"label": "irony",  
"phenomena": ["Lexical Choice", "Hyperbole", "Sarcasm"]  

"text": "The movie was so bad, I could barely keep my eyes open. A masterpiece, truly.",  
"label": "irony",  
"phenomena": ["Polarity Reversal", "Sarcasm", "Contextual Incongruity"]  

"text": "He’s the best driver I know. He’s never gotten into an accident... oh wait, he has."  
"label": "irony",  
"phenomena": ["Contrastive Conjunctions", "Contextual Incongruity", "Sarcasm"]  

"text": "Sure, because skipping breakfast is such a great idea for energy."  
"label": "irony",  
"phenomena": ["Sarcasm", "Contextual Incongruity"]  

"text": "I’m excited to try this new restaurant tonight. It’s my favorite place to eat.",  
"label": "non-irony",  
"phenomena": ["Lexical Choice"]  

"text": "She finished her work on time, and it was a great accomplishment for the team.",  
"label": "non-irony",  
"phenomena": ["Contextual Incongruity"]  

"text": "Sure, skipping breakfast is a great idea for boosting energy!",  
"label": "irony",  
"phenomena": ["Sarcasm", "Contextual Incongruity"]  

"text": "The meeting went well, and we made great progress on the project.",  
"label": "non-irony",  
"phenomena": ["Lexical Choice"]  